In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed
import sys
sys.path.append("../")
import functions as f
plt.style.use('seaborn-v0_8-whitegrid')
#plt.style.available
import seaborn as sns
import load_data as ld
import classes as cl
from sklearn import datasets
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Own code regression

In [2]:
# Load data for regression
X, y, x, a_true = ld.load_simple_data(N_samples=100, noise=0.1)

# Reshape target variable
y = y.reshape(-1, 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
X_train, X_test = f.scale_train_test(X_train, X_test)
y_train, y_test = f.scale_train_test(y_train, y_test)

"""scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)"""

# Set layer sizes: 3 (input), 12 (hidden), 12 (hidden), 1 (output)
input_size = X_train.shape[1]  # 3 input features (ones, x, x**2)
layer_sizes = [12, 12, 1]

# Initialize layers and activation functions
layers = f.create_layers_batch(input_size, layer_sizes)
print("Initialized layers:", layers)

# ReLU for hidden layers, Linear for the output layer
activation_funcs = [f.ReLU, f.ReLU, f.linear]

# Train the neural network
learning_rate = 0.001
epochs = 1000
trained_layers = f.train_network(X_train, y_train, layers, activation_funcs, f.cost_mse, learning_rate, epochs)

# Predict on test data
y_pred = f.feed_forward_batch(X_test, trained_layers, activation_funcs)

# Calculate performance metrics
mse_value = f.mse(y_test, y_pred)
r2_value = f.r2(y_test, y_pred)

# Output results
print(f"MSE: {mse_value:.4f}")
print(f"R^2: {r2_value:.4f}")

Initialized layers: [(array([[-1.22541231,  1.12483019, -1.18774191],
       [ 0.77541339, -0.20738344,  0.35320785],
       [-1.32318179, -1.68296507,  0.68806332],
       [-1.08474263,  0.72047135, -0.10247049],
       [-1.52140562, -0.81303562,  0.9188144 ],
       [-1.26100016, -1.0765001 , -1.02462687],
       [-1.00074295,  0.79076176, -1.15059464],
       [-0.43591634,  0.69940835, -1.44766331],
       [ 0.36806041, -0.49217466,  0.31620462],
       [-0.7567379 ,  0.13048796,  1.90650565],
       [-0.62790637, -1.62144179,  0.20250251],
       [-0.72019269, -0.37713335,  0.29725673]]), array([-0.49410627, -0.54833848,  1.1588893 ,  0.00366215, -0.74335484,
       -0.08281848, -0.01498733,  1.4427861 ,  0.17804021, -0.65711604,
       -1.01366502,  2.56251305])), (array([[-1.30788808e+00, -9.91688806e-01,  2.43322973e-02,
        -1.50533200e+00, -9.05182349e-01, -7.92345613e-02,
         8.22456443e-02,  1.01779826e-01, -2.28292817e-01,
         3.23220194e-01,  3.68581598e-01, 

## Keras code regression

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim


# Main function
def main_regression():
    # Load data for regression
    X, y, _, _ = ld.load_simple_data(N_samples=100, noise=0.1)
    
    # Reshape target variable
    y = y.reshape(-1, 1)

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert data to torch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    # Set model parameters
    input_size = X_train.shape[1]
    hidden_size1 = 12
    hidden_size2 = 12
    output_size = 1

    # Initialize the model, loss function, and optimizer
    model = cl.RegClasNN(input_size, hidden_size1, hidden_size2, output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    num_epochs = 1000
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)

    # Convert PyTorch tensors to NumPy arrays for metric calculation
    y_test_np = y_test.detach().numpy()
    y_pred_np = y_pred.detach().numpy()

    # Calculate performance metrics
    mse_value = f.mse(y_test_np, y_pred_np)
    r2_value = f.r2(y_test_np, y_pred_np)

    # Output results
    print(f"MSE: {mse_value:.4f}")
    print(f"R^2: {r2_value:.4f}")

if __name__ == "__main__":
    main_regression()


Epoch [100/1000], Loss: 2.1030
Epoch [200/1000], Loss: 1.9072
Epoch [300/1000], Loss: 1.8879
Epoch [400/1000], Loss: 1.8817
Epoch [500/1000], Loss: 1.8754
Epoch [600/1000], Loss: 1.8706
Epoch [700/1000], Loss: 1.8683
Epoch [800/1000], Loss: 1.8670
Epoch [900/1000], Loss: 1.8662
Epoch [1000/1000], Loss: 1.8657
MSE: 1.5424
R^2: -0.5867


## Own code classification

In [4]:
# Prepare the dataset
data = pd.read_csv('../../data/Wisconsin.csv')

# Select features and target (radius_mean and texture_mean, diagnosis)
X = data[['radius_mean', 'texture_mean']].values
y = data['diagnosis'].apply(lambda x: 1 if x == 'M' else 0).values.reshape(-1, 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Standardize the data. It does not make sense to standardize/scale the y values because they are binary.
X_train, X_test = f.scale_train_test(X_train, X_test)

# Set layer sizes: 2 (input), 16 (hidden), 8 (hidden), 1 (output)
input_size = 2
layer_sizes = [12, 12, 1]

# Initialize layers and activation functions
layers = f.create_layers_batch(input_size, layer_sizes)
print("Initialized layers:", layers)

activation_funcs = [f.ReLU, f.ReLU, f.sigmoid]

# Train the neural network
learning_rate = 0.001
epochs = 1000
trained_layers = f.train_network(X_train, y_train, layers, activation_funcs, f.cost_cs, learning_rate, epochs)

# Predict on test data
y_pred = f.feed_forward_batch(X_test, trained_layers, activation_funcs)
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate accuracy
accuracy = np.mean(y_pred_classes == y_test)
print(f"Accuracy: {accuracy:.4f}")

Initialized layers: [(array([[-1.22541231,  1.12483019],
       [-1.18774191,  0.77541339],
       [-0.20738344,  0.35320785],
       [-1.32318179, -1.68296507],
       [ 0.68806332, -1.08474263],
       [ 0.72047135, -0.10247049],
       [-1.52140562, -0.81303562],
       [ 0.9188144 , -1.26100016],
       [-1.0765001 , -1.02462687],
       [-1.00074295,  0.79076176],
       [-1.15059464, -0.43591634],
       [ 0.69940835, -1.44766331]]), array([ 0.36806041, -0.49217466,  0.31620462, -0.7567379 ,  0.13048796,
        1.90650565, -0.62790637, -1.62144179,  0.20250251, -0.72019269,
       -0.37713335,  0.29725673])), (array([[-4.94106268e-01, -5.48338476e-01,  1.15888930e+00,
         3.66214897e-03, -7.43354844e-01, -8.28184848e-02,
        -1.49873302e-02,  1.44278610e+00,  1.78040210e-01,
        -6.57116036e-01, -1.01366502e+00,  2.56251305e+00],
       [-1.30788808e+00, -9.91688806e-01,  2.43322973e-02,
        -1.50533200e+00, -9.05182349e-01, -7.92345613e-02,
         8.22456443e

## Pytorch code classification

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd

# Main function
def main_classification():
    # Load data for classification
    X, y = ld.load_wisconsin_data()

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert data to torch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    # Set model parameters
    input_size = X_train.shape[1]
    hidden_size1 = 12
    hidden_size2 = 12
    output_size = 1

    # Initialize the model, loss function, and optimizer
    model = cl.RegClasNN(input_size, hidden_size1, hidden_size2, output_size)
    criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    num_epochs = 1000
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        y_pred_classes = (y_pred > 0.5).float()

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred_classes.numpy())
    print(f"Accuracy: {accuracy:.4f}")

if __name__ == "__main__":
    main_classification()


Epoch [100/1000], Loss: 0.4431
Epoch [200/1000], Loss: 0.2982
Epoch [300/1000], Loss: 0.2633
Epoch [400/1000], Loss: 0.2543
Epoch [500/1000], Loss: 0.2497
Epoch [600/1000], Loss: 0.2464
Epoch [700/1000], Loss: 0.2439
Epoch [800/1000], Loss: 0.2421
Epoch [900/1000], Loss: 0.2407
Epoch [1000/1000], Loss: 0.2396
Accuracy: 0.9123
